# ЛЕГЕНДА

Итак, мы работаем аналитиком в онлайн-школе MasterMind.

Уже в конце рабочего дня нам пишет расстроенный продакт-менеджер. Несчастный Григорий крайне устал от того, что новые курсы, созданные с той же любовью, что и прежние, не пользуются особой популярностью среди пользователей — несмотря на все усилия отдела маркетинга. 

# ЦЕЛЬ
Подготовить основу рекомендательной системы.

# ЗАДАЧИ

1. Подготовить рекомендательную систему, благодаря которой можно будет предлагать клиентам интересные им курсы и тем самым повышать средний чек.
2. Данные предоставить в таблице, в которой курсам будет соответствовать по две рекомендации.
3. Все коды снабдитькомментариями, что где и почему мы делаем.
4. Проанализировать результаты A/B-теста


# КОНКРЕТНЫЕ ШАГИ (ФОРМАЛИЗОВАННЫЕ ЗАДАЧИ)

Действуем по схеме:

1. Познакомимся с датасетом, подготовим и проанализируем данные с помощью SQL.
2. Обработаем данные средствами Python.
3. Составим итоговую таблицу с рекомендациями, снабдив её необходимыми комментариями, и представим отчёт продакт-менеджеру.
4. Проанализируем результаты A/B-теста, проведённого после внедрения фичи, и сделаем вывод.

# РЕЗУЛЬТАТ ПРОЕКТА

Итогом работы станет файл, содержащий результаты всех промежуточных этапов: скрипты с комментариями, таблица рекомендаций и выводы.

Импортируем библиотеки

In [314]:
import pandas as pd
import numpy as np
import psycopg2
import psycopg2.extras 
import itertools
from collections import Counter

## Изучим данные


Посмотрим, за какие годы есть продажи.  

In [315]:
# Код на SQL:
a = """
SELECT 
   min(updated_at),
   max(updated_at)
FROM final.carts 
WHERE state = 'successful'
"""

In [316]:
def getQuerySQL1():
    query = a.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data

date_salse = pd.DataFrame(getQuerySQL1())
date_salse

,min,max
0,2017-01-01 07:29:15.159,2018-12-30 23:29:38.889


Посмотрим, сколько клиентов покупали курсы.  

In [317]:
# Код на SQL:
b = """
SELECT 
    count (distinct user_id) as users_count
FROM final.carts fc
LEFT JOIN final.cart_items fci ON fc.id = fci.cart_id
WHERE fci.resource_type = 'Course' AND fc.state = 'successful'
"""

In [318]:
def getQuerySQL2():
    query = b.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data

users_count = getQuerySQL2()
users_count

[{'users_count': 49006}]

In [319]:
# Код на SQL:
c = """
SELECT 
    count (distinct fci.resource_id) as course_count
FROM final.carts fc
LEFT JOIN final.cart_items fci ON fc.id = fci.cart_id
WHERE fci.resource_type = 'Course' 
"""

In [320]:
def getQuerySQL3():
    query = c.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data

course_count = getQuerySQL3()
course_count

[{'course_count': 127}]

Каково среднее число купленных курсов на одного клиента?

In [321]:
# Код на SQL:
d = """
WITH join_table as
    (
    SELECT 
       user_id,
       count(user_id) as count_course
    FROM final.carts fc
    LEFT JOIN final.cart_items fci ON fc.id = fci.cart_id
    WHERE fci.resource_type = 'Course' AND fc.state = 'successful'
    GROUP BY 1
    )
SELECT 
    avg(count_course) as avg_count_course
FROM join_table
"""

In [322]:
def getQuerySQL4():
    query = d.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data

avg_count_course = getQuerySQL4()
avg_count_course

[{'avg_count_course': Decimal('1.4402522140146105')}]

Сколько клиентов купили больше одного курса?

In [323]:
# Код на SQL:
e = """
WITH data
AS (
	SELECT DISTINCT 
	    user_id,
		resource_id
	FROM final.carts AS c
	INNER JOIN final.cart_items AS i ON c.id = i.cart_id
	WHERE state = 'successful' AND resource_type = 'Course'
	)
	,counter
AS (
	SELECT 
	    user_id,
		count(resource_id) AS cc
	FROM data
	GROUP BY user_id
	)
SELECT 
    count(user_id) as users_purchased_more_one_course
FROM counter
WHERE cc > 1
"""

In [324]:
def getQuerySQL5():
    query = e.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data

users_purchased_more_one_course = getQuerySQL5()
users_purchased_more_one_course

[{'users_purchased_more_one_course': 12656}]

Подготовим датафрейм с данными по продажам курсов в разрезе пользователей (купивших более одного курса):

In [325]:
# Код на SQL:
f = """
WITH 
data AS -- таблица уникальных пар: пользователь курс
    (
	SELECT DISTINCT
	    user_id,
		resource_id
	FROM final.carts AS c
	INNER JOIN final.cart_items AS i ON c.id = i.cart_id
	WHERE state = 'successful' AND resource_type = 'Course'
	),
users AS -- пользователи, купившие более одного курса
    (
	SELECT 
        user_id
	FROM data
	GROUP BY user_id
	HAVING 	count(resource_id) > 1
	)
SELECT -- таблицица с информацией по купленным курсам для пользователей купивших более 1 курса
    fc.id as cart_id,
    fc.created_at,
    fci.created_at as set_course_at,
    fc.updated_at,
    fc.user_id,
    fc.promo_code_id,
    fci.resource_id
FROM final.carts fc
INNER JOIN final.cart_items fci ON fc.id = fci.cart_id
WHERE fci.resource_type = 'Course' 
    AND fc.state = 'successful'
    AND fc.user_id IN (SELECT user_id FROM users)
ORDER BY fc.user_id, fc.updated_at  
"""

In [326]:
def getQuerySQL6():
    query = f.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data

df = pd.DataFrame(getQuerySQL6())


Посмотрим, сколько клиентов покупали курсы.  

In [327]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34172 entries, 0 to 34171
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   cart_id        34172 non-null  int64         
 1   created_at     34172 non-null  datetime64[ns]
 2   set_course_at  34172 non-null  datetime64[ns]
 3   updated_at     34172 non-null  datetime64[ns]
 4   user_id        34172 non-null  int64         
 5   promo_code_id  2580 non-null   float64       
 6   resource_id    34172 non-null  int64         
dtypes: datetime64[ns](3), float64(1), int64(3)
memory usage: 1.8 MB


Как видим, все данные в нужном формате, включая даты

In [328]:
df.head()

,cart_id,created_at,set_course_at,updated_at,user_id,promo_code_id,resource_id
0,205876,2016-12-04 17:06:44.645,2016-12-04 17:06:44.656,2017-01-06 21:31:53.504,51,NaN,516
1,475936,2018-06-22 17:18:04.386,2018-06-22 17:18:04.444,2018-06-22 17:20:49.078,51,NaN,1099
2,295889,2017-06-28 08:55:13.920,2017-06-28 13:13:37.989,2017-06-30 17:36:47.872,6117,NaN,357
3,295889,2017-06-28 08:55:13.920,2017-06-28 08:55:13.939,2017-06-30 17:36:47.872,6117,NaN,356
4,487524,2018-08-01 04:56:53.871,2018-08-01 04:56:53.923,2018-08-01 05:01:45.028,6117,NaN,1125


## Обработаем данные

Разбьем все покупки курсов по парам, после чего проранжируем их для каждого курса.

In [329]:
courses = df['resource_id'].unique() # список уникальных курсов

In [330]:
# по каждому пользователю выводим список курсов
user_df = df.groupby('user_id')['resource_id'].apply(lambda x: list(np.unique(x))).reset_index()
user_df.head()

,user_id,resource_id
0,51,"[516, 1099]"
1,6117,"[356, 357, 1125]"
2,10275,"[553, 1147]"
3,10457,"[361, 1138]"
4,17166,"[356, 357]"


In [331]:
# создаем список пар курсов
list_course = list()
for resource in user_df['resource_id']:
    for pair_course in itertools.combinations(resource,2):
        list_course.append(pair_course)

In [332]:
# расчитываем количество пар курсов
pair_course_count = Counter([pair_course for pair_course in list_course])
pair_course_count

Counter({(516, 1099): 25,
         (356, 357): 100,
         (356, 1125): 44,
         (357, 1125): 52,
         (553, 1147): 16,
         (361, 1138): 40,
         (1125, 1140): 1,
         (551, 745): 138,
         (553, 745): 212,
         (551, 1138): 14,
         (553, 568): 83,
         (514, 517): 10,
         (514, 566): 138,
         (517, 566): 21,
         (363, 511): 99,
         (363, 562): 77,
         (363, 563): 33,
         (511, 562): 55,
         (511, 563): 19,
         (562, 563): 53,
         (568, 745): 102,
         (509, 553): 48,
         (509, 745): 59,
         (1125, 1144): 22,
         (509, 568): 46,
         (509, 672): 5,
         (568, 672): 4,
         (516, 552): 12,
         (356, 552): 7,
         (357, 571): 112,
         (509, 516): 35,
         (516, 568): 54,
         (513, 1141): 34,
         (571, 1125): 122,
         (551, 552): 177,
         (551, 744): 16,
         (551, 862): 8,
         (552, 744): 8,
         (552, 862): 6,
         (55

In [333]:
# посчитаем сколько различных пар курсов встречаются вместе в покупках клиентов
len(pair_course_count)

3989

In [334]:
# сортируем словарь по количеству покупок по убыванию
sorted_pair_course = {y: a for y, a in sorted(pair_course_count.items(), key=lambda iteam: iteam[1], reverse=True)}
sorted_pair_course

{(551, 566): 797,
 (515, 551): 417,
 (489, 551): 311,
 (523, 551): 304,
 (566, 794): 290,
 (489, 515): 286,
 (490, 566): 253,
 (490, 551): 247,
 (570, 752): 247,
 (569, 572): 216,
 (515, 523): 213,
 (553, 745): 212,
 (489, 523): 206,
 (569, 840): 204,
 (514, 551): 200,
 (516, 745): 199,
 (515, 566): 195,
 (489, 566): 188,
 (504, 572): 184,
 (572, 840): 178,
 (551, 552): 177,
 (507, 570): 172,
 (490, 809): 163,
 (489, 490): 152,
 (507, 752): 150,
 (523, 552): 144,
 (490, 515): 143,
 (551, 570): 142,
 (504, 569): 139,
 (514, 515): 139,
 (551, 745): 138,
 (514, 566): 138,
 (502, 551): 135,
 (504, 840): 135,
 (571, 1125): 122,
 (502, 566): 120,
 (523, 566): 120,
 (570, 809): 119,
 (752, 809): 115,
 (490, 523): 114,
 (357, 571): 112,
 (523, 564): 110,
 (551, 749): 109,
 (516, 553): 107,
 (551, 777): 107,
 (551, 679): 104,
 (356, 571): 103,
 (551, 564): 103,
 (515, 749): 103,
 (568, 745): 102,
 (356, 357): 100,
 (363, 511): 99,
 (551, 571): 98,
 (551, 809): 96,
 (502, 514): 95,
 (551, 794): 

In [335]:
# Найдем самую популярную пару курсов
a = list(sorted_pair_course.keys())[0]
a

(551, 566)

In [336]:
# расчитаем квартили по количеству покупок
quantile_df = pd.DataFrame({'count': sorted_pair_course.values()})
quantile_df['count'].quantile([0.25,0.5,0.75])

0.25    1.0
0.50    3.0
0.75    9.0
Name: count, dtype: float64

Пары, которые купили 1 раз составляют четверть всех данных, предлагаем из расчета рекомендаций их исключить, и заменить на самую популярную пару.

In [337]:
# сортируем словарь по первому id курса в паре по возрастанию
sorted_pair_course = {y: a for y, a in sorted(pair_course_count.items(), key=lambda iteam: iteam[0], reverse=False)}
sorted_pair_course

{(356, 357): 100,
 (356, 360): 1,
 (356, 361): 17,
 (356, 366): 15,
 (356, 367): 12,
 (356, 368): 1,
 (356, 489): 26,
 (356, 490): 13,
 (356, 502): 17,
 (356, 503): 1,
 (356, 508): 1,
 (356, 509): 5,
 (356, 513): 1,
 (356, 514): 35,
 (356, 515): 21,
 (356, 516): 16,
 (356, 517): 3,
 (356, 519): 14,
 (356, 523): 24,
 (356, 551): 48,
 (356, 552): 7,
 (356, 553): 5,
 (356, 564): 3,
 (356, 566): 21,
 (356, 568): 6,
 (356, 569): 2,
 (356, 570): 1,
 (356, 571): 103,
 (356, 659): 4,
 (356, 671): 2,
 (356, 672): 4,
 (356, 679): 8,
 (356, 742): 1,
 (356, 743): 1,
 (356, 745): 8,
 (356, 749): 1,
 (356, 750): 3,
 (356, 753): 3,
 (356, 756): 5,
 (356, 757): 1,
 (356, 764): 8,
 (356, 765): 35,
 (356, 776): 2,
 (356, 777): 4,
 (356, 791): 1,
 (356, 794): 10,
 (356, 800): 3,
 (356, 803): 1,
 (356, 809): 2,
 (356, 829): 5,
 (356, 835): 1,
 (356, 840): 1,
 (356, 862): 3,
 (356, 863): 1,
 (356, 866): 1,
 (356, 907): 1,
 (356, 908): 3,
 (356, 909): 3,
 (356, 912): 34,
 (356, 1099): 3,
 (356, 1100): 5,
 (

In [338]:
# удаляем из списка пары, встречающиеся 1 раз
sorted_pair_course_corr = {key:val for key, val in sorted_pair_course.items() if val <= 1}
sorted_pair_course_corr

{(356, 360): 1,
 (356, 368): 1,
 (356, 503): 1,
 (356, 508): 1,
 (356, 513): 1,
 (356, 570): 1,
 (356, 742): 1,
 (356, 743): 1,
 (356, 749): 1,
 (356, 757): 1,
 (356, 791): 1,
 (356, 803): 1,
 (356, 835): 1,
 (356, 840): 1,
 (356, 863): 1,
 (356, 866): 1,
 (356, 907): 1,
 (356, 1102): 1,
 (356, 1116): 1,
 (356, 1124): 1,
 (356, 1139): 1,
 (356, 1187): 1,
 (357, 362): 1,
 (357, 503): 1,
 (357, 504): 1,
 (357, 511): 1,
 (357, 569): 1,
 (357, 749): 1,
 (357, 757): 1,
 (357, 813): 1,
 (357, 835): 1,
 (357, 840): 1,
 (357, 862): 1,
 (357, 863): 1,
 (357, 907): 1,
 (357, 1101): 1,
 (357, 1116): 1,
 (357, 1128): 1,
 (357, 1147): 1,
 (357, 1152): 1,
 (357, 1156): 1,
 (357, 1198): 1,
 (358, 361): 1,
 (358, 490): 1,
 (358, 511): 1,
 (358, 517): 1,
 (358, 518): 1,
 (358, 564): 1,
 (358, 755): 1,
 (358, 777): 1,
 (358, 840): 1,
 (358, 864): 1,
 (358, 908): 1,
 (358, 1103): 1,
 (358, 1152): 1,
 (358, 1181): 1,
 (359, 363): 1,
 (359, 508): 1,
 (359, 517): 1,
 (359, 564): 1,
 (359, 659): 1,
 (359, 67

In [339]:
def recommend(course):
    course_list = [] # создаем пустой список
    for i in sorted_pair_course_corr.keys(): # перебираем ключи словаря
        if i[0] == course: # проверяем, является ли первый элемент пары курсом
           course_list.append((i, sorted_pair_course_corr[i])) # добавляем в созданный список саму пару и ее значение
    sorted_course_list = sorted(course_list, key=lambda x: x[1], reverse=True) # сортируем значения в обратном порядке
    return sorted_course_list[:2]
            

In [340]:
recommend_df = pd.DataFrame(columns=['Course', 'Recommendation_1','Recommendation_2']) # создаем датафрейм, куда будем вносить данные
for i in courses: # перебираем список всех курсов
    if len(recommend(i)) == 2: # применяем функцию recommend(), и если выдает 2 рекомендации для курса, записываем их в таблицу
        recommend_df.loc[i] = [i, recommend(i)[0][0][1], recommend(i)[1][0][1]]
    elif  len(recommend(i)) == 1: # применяем функцию recommend(), и если выдает 1 рекомендацию то записываем ее в качестве 2-ой, а в качестве 1-ой записываем самый популярный курс
        recommend_df.loc[i] = [i, a[0], recommend(i)[0][0][1]]   
    elif  len(recommend(i)) == 0:
        recommend_df.loc[i] = [i, a[0], a[1]]  # если ничего не выводит, ставим самые популярные курсы 

In [341]:
recommend_df = recommend_df.sort_values(by = 'Course')
recommend_df

,Course,Recommendation_1,Recommendation_2
356,356,360,368
357,357,362,503
358,358,361,490
359,359,363,508
360,360,361,365
...,...,...,...
1188,1188,551,566
1198,1198,551,566
1199,1199,551,566
1200,1200,551,566


Получаем таблицу рекомендаций по популярности курсов по данным пользователей
Для курсов, которые покупались не более 1 раза предлагаем самую популярную пару курсов!

https://colab.research.google.com/drive/1ZJMhefdj_XCZbbrIFDv29glrv6JTEQB7?usp=sharing